In [30]:
import json 
import pickle as pkl
from scipy.io import loadmat
import numpy as np
print("Reloading modules")
%load_ext autoreload
%autoreload 2

Reloading modules
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
# Data that is actally used by RNNTraining -> Our target format.
small_dbtrain = loadmat('./coco/dbval_small.mat')
db = small_dbtrain

In [90]:
# DBTRAIN: What we must produce.
num = 0 # Index of the image in the list.
print("Type: " + str(type(small_dbtrain)))
print("Number of keys: " + str(len(small_dbtrain.keys())))
print("An example of key:" + str(small_dbtrain.keys()[0]))
print("Type of element " + str(num) +" :" + str(type(small_dbtrain[small_dbtrain.keys()[num]])))
print("Shape of element " + str(num) +" :" + str(np.shape(small_dbtrain[small_dbtrain.keys()[num]])))
one_element = small_dbtrain[small_dbtrain.keys()[num]][0][0]
print("One value (one key):")
print(one_element)
print("First element: " + str(one_element[0]))
print("Shape of the second element: " + str(np.shape(one_element[1])))
print(one_element[1][0,:])
# GOAL: a dict with image_id as key containing a tuple with (sentences, numpy array)

Type: <type 'dict'>
Number of keys: 1003
An example of key:9077
Type of element 0 :<type 'numpy.ndarray'>
Shape of element 0 :(1, 1)
One value (one key):
(array([ u'A variety of fruit is displayed in a market.                                           ',
       u'Fruits and vegetables for sale in a farmers market.                                    ',
       u'Many fruits displayed in a produce store including bananas, oranges, apples and lemons.',
       u'The produce section of a grocery store showing many various fruits.                    ',
       u'A market has many fruits and vegetables out for display.                               '],
      dtype='<U87'), array([[ 172.,   45.,  640.,  427.],
       [ 197.,   62.,  540.,  427.],
       [  22.,   45.,  589.,  427.],
       ..., 
       [ 135.,  265.,  160.,  359.],
       [ 285.,   61.,  320.,   96.],
       [  66.,  186.,  124.,  244.]], dtype=float32), array([[255, 216, 255, ..., 143, 255, 217]], dtype=uint8))
First element: [

In [78]:
# Prying that resembles the one in RNNTraining
#val = db.iteritems().next()[1]

In [43]:
flickdata = json.load(open('./flickr30k/flickr30k/dataset.json'))

In [56]:
# Analysis of what we have as data
num=0
print(type(flickdata))
print("Keys: " + str(flickdata.keys()))
flick_images = flickdata['images']
print("Number of images in List: " + str(len(flick_images)))
print("Keys for one image: " + str(flick_images[num].keys()))
print("Split for an image: " + str(flick_images[num]['split']))


<type 'dict'>
Keys: [u'images', u'dataset']
Number of images in List: 31014
Keys for one image: [u'filename', u'imgid', u'sentences', u'split', u'sentids']
Split for an image: train


In [62]:
# Taking a look at flickr matrix, let's see if maybe that's what we want
vgg_flick = loadmat('./flickr30k/flickr30k/vgg_feats.mat')

In [75]:
print("Type of vgg_flick: " + str(type(vgg_flick)))
print("Keys: " + str(vgg_flick.keys()))
#vgg_feats = {key: vgg_flick[key]['feats'] for key in vgg_flick.keys()}
try:
    del vgg_flick['__version__']
    del vgg_flick['__header__']
    del vgg_flick['__globals__']
except:
    print("Keys must have been deleted already")
vgg_feats = vgg_flick['feats']
print(np.shape(vgg_feats))
print("Conclusion: If we decide to trust it, vgg_flick seems to contain the features we want.")

Type of vgg_flick: <type 'dict'>
Keys: ['feats']
Keys must have been deleted already
(4096, 31014)


In [2]:
#annotations AND IMAGES used with COCO.
ann_path = "coco/annotations/"
split = 'train'
ann=json.load(open(ann_path+'captions_%s2014.json'%split))
print(ann.keys())
annotations=ann['annotations']

[u'info', u'images', u'licenses', u'annotations']


In [7]:
print(type(annotations))
print(ann['images'][0])

<type 'list'>
{u'license': 5, u'file_name': u'COCO_train2014_000000057870.jpg', u'coco_url': u'http://mscoco.org/images/57870', u'height': 480, u'width': 640, u'date_captured': u'2013-11-14 16:28:13', u'flickr_url': u'http://farm4.staticflickr.com/3153/2970773875_164f0c0b83_z.jpg', u'id': 57870}


In [5]:
print("Number of annotations: " + str(len(annotations))) # More than one annotation per image..
print(annotations[0].keys()) 
print(annotations[0]['caption']) 
print(annotations[0]['image_id']) 
print(annotations[0]['id'])  

Number of annotations: 414113
[u'image_id', u'id', u'caption']
A very clean and well decorated empty bathroom
318556
48


In [ ]:
#Analysis of captions produced when training on coco.

In [31]:
# Analysis of annotations provided with the flick
info = json.load(open('./flickr30k/flickr30k/dataset.json'))
print(info.keys())
informations = info['images']
print(info['dataset'])

[u'images', u'dataset']
flickr30k


In [32]:
print(informations[0].keys())
info = informations[0]
print(type(info['imgid']))
print(info['filename'])

[u'filename', u'imgid', u'sentences', u'split', u'sentids']
<type 'int'>
1000092795.jpg


In [10]:
# Analysis of annotations.dict found in flickr30k: maybe Marco did it already?
annot =  pkl.load(open('/scratch/clear/mpederso/git/ImCap/flickr30k/annotations.dict'))


In [33]:
print(type(annot))
print(len(annot.keys()))
k = annot.keys()[0]
print(annot[k].keys())
print(annot[k]['sentence'][0])
print(annot[k]['idToLabel'][0][0][0])
print(annot[k]['id'][0][0])
print(annot[k]['id'][0][0])

NameError: name 'annot' is not defined

In [55]:
# Creating the 'annotations' and 'images' fields.
my_ann, my_im, count = [], [], 0
for i in informations:
    #my_im.append({'file_name': i['filename'], 'id': int(i['imgid']), 'split': i['split']})
    my_im.append({'file_name': i['filename'], 'id': int(i['filename'].split('.')[0]), 'split': i['split']})
    for sent in i['sentences']:
       #my_ann.append({'image_id': int(i['imgid']) , 'id': count , 'caption': sent, 'split': i['split']}) 
       my_ann.append({'image_id': int(i['filename'].split('.')[0]), 'id': count , 'caption': sent['raw'], 'split': i['split']}) 
       count += 1
assert(len(my_ann) == 5*len(informations)), "Unexpected length..."

In [56]:
# Creating the 'images' field of my_ann.
print(len(informations))
print(informations[1].keys())
print(informations[1]['imgid'])
print(informations[1]['filename'])


31014
[u'filename', u'imgid', u'sentences', u'split', u'sentids']
1
10002456.jpg


In [57]:
train_ann, val_ann, test_ann = {}, {}, {}
train_ann['annotations'] = [e for e in my_ann if e['split'] == 'train']
train_ann['images'] = [e for e in my_im if e['split'] == 'train']
val_ann['annotations'] = [e for e in my_ann if e['split'] == 'val']
val_ann['images'] = [e for e in my_im if e['split'] == 'val']
test_ann['annotations'] = [e for e in my_ann if e['split'] == 'test']
test_ann['images'] = [e for e in my_im if e['split'] == 'test']
end_len = len(train_ann['annotations']) + len(val_ann['annotations']) + len(test_ann['annotations']) 
assert(end_len == len(my_ann)), "Lost annotation: " + str(end_len) + " / " + str(len(my_ann))
#assert(len(train_im) + len(val_im) + len(test_im) == len(my_im)), "Lost images...."

In [58]:
print(len(val_ann['images']))

1014


In [59]:
with open('./flick_data/train_annot.json', 'w') as outfile:
    json.dump(train_ann, outfile)
print("1/3 ..."),
with open('./flick_data/val_annot.json', 'w') as outfile:
    json.dump(val_ann, outfile)
print("2/3 ..."),
with open('./flick_data/test_annot.json', 'w') as outfile:
    json.dump(test_ann, outfile)
print("3/3 OK")

1/3 ... 2/3 ... 3/3 OK


In [60]:
# Debugging caption evaluation.
captionfile = json.load(open('./flick_data/debug_delme.json'))
from CaptionEvaluation import evaluateCaptionsFlickr

In [61]:
res = evaluateCaptionsFlickr(captionfile)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Loading and preparing results...     
DONE (t=0.01s)
creating index...
index created!
tokenization...
> /scratch/algorab/tlucas/save_new_imcap/newpull_imccap/imcap/coco-caption/pycocoevalcap/eval.py(38)evaluate()
-> gts  = tokenizer.tokenize(gts)
(Pdb) c
setting up scorers...
computing Bleu score...
{'reflen': 10033, 'guess': [10140, 9126, 8112, 7098], 'testlen': 10140, 'correct': [4503, 1055, 133, 10]}
ratio: 1.01066480614
Bleu_1: 0.444
Bleu_2: 0.227
Bleu_3: 0.094
Bleu_4: 0.033
computing METEOR score...
METEOR: 0.102
computing Rouge score...
ROUGE_L: 0.314
computing CIDEr score...
CIDEr: 0.020


In [21]:
## Check the training files to see if id and file names are the same:
dbtrain = pkl.load(open('./flick_data/dbtrain.pkl'))

In [27]:
k = dbtrain.keys()[0]
print(k)
print(dbtrain[k].keys())
print(dbtrain[k]['caption'])

145558685
['caption', 'image']
[u'A young girl with a big smile, wearing brightly colored clothing helps prepare food in a narrow alley.', u'A girl wearing a blue floral patterned dress is sitting on the cement and sorting leaves.', u'This is a street in a poor area where two people are sorting through tobacco.', u'This lady in the print dress is stripping food for dinner.', u'A woman puts the firewood in order.']
